# Sistema de Recuperación Multimodal de Información para E-Commerce

**Proyecto de Recuperación de Información - 2do Bimestre**

Este notebook implementa un sistema completo de búsqueda multimodal con:
- Búsqueda texto → productos
- Búsqueda imagen → productos  
- Re-ranking de resultados
- Generación aumentada por recuperación (RAG)
- Búsqueda conversacional con contexto

## 1. Instalación de Dependencias

Instalación de librerías base para el procesamiento de visión por computadora (torch, torchvision), procesamiento de lenguaje natural (sentence-transformers) y gestión de bases de datos vectoriales (chromadb). También se incluyen herramientas para la creación de interfaces rápidas (gradio) y la comunicación con la API de Kaggle.

In [25]:
# MODELOS DE DEEP LEARNING
!pip install -q torch torchvision ftfy regex tqdm
# torch: Framework de deep learning principal (PyTorch)

# MODELO MULTIMODAL CLIP
!pip install -q git+https://github.com/openai/CLIP.git
# CLIP: Contrastive Language-Image Pre-training (OpenAI)

# RE-RANKING
!pip install -q sentence-transformers
# sentence-transformers: Framework para embeddings de texto semánticos

# BASE DE DATOS VECTORIAL
!pip install -q chromadb
# chromadb: Base de datos vectorial embeddings-first

# INTERFAZ DE USUARIO
!pip install -q gradio
# gradio: Framework para crear interfaces web interactivas

# PROCESAMIENTO DE IMÁGENES
!pip install -q Pillow
# Pillow: Python Imaging Library (PIL)

# MANIPULACIÓN DE DATOS
!pip install -q pandas
# pandas: Librería de análisis de datos

# DESCARGA DE DATASETS
!pip install -q kaggle


  Preparing metadata (setup.py) ... done


## 2. Configuración de Kaggle y Descarga del Dataset

Autenticación con Kaggle mediante el archivo de credenciales kaggle.json. El objetivo es automatizar la descarga y descompresión del dataset de Amazon directamente en el almacenamiento temporal de la sesión de Colab

In [2]:
# Subir Archivo kaggle.json
from google.colab import files
import os

print("Archivo kaggle.json")
uploaded = files.upload()

# Configurar credenciales de Kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("Credenciales de Kaggle configuradas")

Archivo kaggle.json


Saving kaggle.json to kaggle.json
Credenciales de Kaggle configuradas


In [8]:
import kagglehub
# Pandas: Librería de análisis de datos
import pandas as pd

# 1. Descarga automática (esto baja el .zip, lo extrae y te da la ruta)
path = kagglehub.dataset_download("datafiniti/consumer-reviews-of-amazon-products")

# 2. Listar archivos para estar seguros del nombre
print("Archivos en la carpeta:", os.listdir(path))

# 3. Cargar el archivo
file_path = os.path.join(path, "Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")
df = pd.read_csv(file_path)
df.head()


Using Colab cache for faster access to the 'consumer-reviews-of-amazon-products' dataset.
Archivos en la carpeta: ['Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv', '1429_1.csv', 'Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv']


,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


## 3. Importaciones y Configuración

Se cargan los módulos necesarios y se configuran los modelos pre-entrenados. Se inicializa CLIP (ViT-B/32) para la generación de embeddings multimodales y un Cross-Encoder especializado en MS-MARCO para las tareas de reordenamiento de relevancia (re-ranking)

In [26]:
# DEEP LEARNING Y MODELOS NEURONALES
import torch
# PyTorch: Framework principal de deep learning
import clip
# CLIP de OpenAI: Modelo multimodal texto-imagen

# MANIPULACIÓN DE DATOS
import numpy as np
# NumPy: Operaciones numéricas y arrays

# PROCESAMIENTO DE IMÁGENES
from PIL import Image
# Python Imaging Library (Pillow)

# BASE DE DATOS VECTORIAL
import chromadb
# ChromaDB: Base de datos para embeddings

# RE-RANKING
from sentence_transformers import CrossEncoder
# Sentence-Transformers: Framework para embeddings semánticos

# INTERFAZ DE USUARIO
import gradio as gr
# Gradio: Creación de UIs web interactivas

# UTILIDADES DEL SISTEMA
import json
# Manejo de JSON

from pathlib import Path
# Manejo moderno de rutas de archivos

import warnings
warnings.filterwarnings('ignore')
# Suprimir warnings molestos (ej: deprecation warnings)

from tqdm import tqdm
import requests
from io import BytesIO


### CONFIGURACIÓN DE HARDWARE

se detecta la disponibilidad de GPU (CUDA)

In [27]:
# Detectar si hay GPU disponible (NVIDIA CUDA)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


### CARGA DE MODELOS

In [28]:
# MODELO CLIP
model, preprocess = clip.load("ViT-B/32", device=device)
print(" Modelo CLIP cargado")

 Modelo CLIP cargado


In [29]:
# MODELO DE RE-RANKING
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
print("Modelo de re-ranking cargado")

Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Modelo de re-ranking cargado


## 4. Preprocesamiento del Corpus

Consiste en la limpieza y estructuración de los datos crudos. Se filtran productos sin información esencial, se gestionan valores nulos y se construye una "descripción enriquecida" que combina el nombre, la categoría y la subcategoría del producto para maximizar la calidad de las búsquedas semánticas.

In [30]:
import pandas as pd
import numpy as np
import re
import os
from collections import defaultdict

def preprocess_amazon_reviews_all_files(path, max_products=5000, min_reviews_per_product=1):
    """
    Preprocesa TODOS los archivos CSV del dataset de Amazon para máxima cobertura.

    Parámetros:
    -----------
    path : str
        Ruta al directorio con los archivos CSV
    max_products : int
        Número máximo de productos únicos a mantener
    min_reviews_per_product : int
        Mínimo de reviews que debe tener un producto para incluirlo

    Retorna:
    --------
    pandas.DataFrame
        DataFrame preprocesado con productos únicos y todas sus reviews
    """

    print("PREPROCESAMIENTO DE MÚLTIPLES ARCHIVOS")
    # ==================== PASO 1: CARGAR TODOS LOS ARCHIVOS ====================
    print("\nPaso 1: Cargando todos los archivos CSV...")

    # Lista de archivos CSV en el directorio
    csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
    print(f"   Archivos encontrados: {csv_files}\n")

    # Cargar y combinar todos los DataFrames
    dataframes = []
    total_rows = 0

    for i, csv_file in enumerate(csv_files, 1):
        file_path = os.path.join(path, csv_file)
        print(f"   [{i}/{len(csv_files)}] Cargando {csv_file}...", end=" ")

        try:
            df_temp = pd.read_csv(file_path, low_memory=False)
            rows = len(df_temp)
            total_rows += rows
            dataframes.append(df_temp)
            print(f"{rows:,} filas")
        except Exception as e:
            print(f"Error: {e}")

    # Combinar todos los DataFrames
    print(f"\n Combinando {len(dataframes)} DataFrames...")
    df_all = pd.concat(dataframes, ignore_index=True)
    print(f" Total combinado: {len(df_all):,} filas")


    # ==================== PASO 2: LIMPIEZA BÁSICA ====================
    print("\nPaso 2: Limpieza básica")

    initial_rows = len(df_all)

    # Eliminar duplicados exactos
    df_all = df_all.drop_duplicates()
    print(f" Duplicados eliminados: {initial_rows - len(df_all):,}")

    # Verificar columnas disponibles
    print(f"\n Columnas disponibles: {df_all.columns.tolist()}")

    # Normalizar nombres de columnas (algunos archivos pueden tener variaciones)
    # Mapeo de posibles nombres alternativos
    column_mapping = {
        'reviews.rating': ['rating', 'reviews.rating', 'review.rating'],
        'reviews.text': ['text', 'reviews.text', 'review.text'],
        'reviews.title': ['title', 'reviews.title', 'review.title'],
        'reviews.username': ['username', 'reviews.username', 'review.username']
    }

    # Aplicar mapeo si es necesario
    for standard_name, alternatives in column_mapping.items():
        if standard_name not in df_all.columns:
            for alt in alternatives:
                if alt in df_all.columns:
                    df_all[standard_name] = df_all[alt]
                    break

    # Identificar columnas críticas
    required_columns = ['name', 'imageURLs']
    optional_columns = ['id', 'brand', 'categories', 'primaryCategories',
                       'reviews.text', 'reviews.rating', 'reviews.title',
                       'reviews.username', 'asins']

    # Verificar que existan las columnas requeridas
    missing_cols = [col for col in required_columns if col not in df_all.columns]
    if missing_cols:
        print(f"  Columnas faltantes: {missing_cols}")
        print("   Intentar columnas alternativas")

    # Eliminar filas sin información crítica
    df_clean = df_all.dropna(subset=['name', 'imageURLs']).copy()
    print(f"Eliminadas {len(df_all) - len(df_clean):,} filas sin nombre o imagen")


    # ==================== PASO 3: PROCESAR IMÁGENES ====================
    print("\nPaso 3: Procesar URLs de imágenes")

    def extract_first_image(image_urls):
        """Extrae la primera URL válida de imágenes"""
        if pd.isna(image_urls):
            return None
        # Las URLs pueden venir separadas por comas o como lista
        urls_str = str(image_urls)
        # Limpiar caracteres extraños
        urls_str = urls_str.replace('[', '').replace(']', '').replace('"', '').replace("'", '')
        urls = urls_str.split(',')

        # Tomar primera URL válida (HTTPS prioritario)
        for url in urls:
            url = url.strip()
            if url.startswith('https://'):
                return url
        # Si no hay HTTPS, aceptar HTTP
        for url in urls:
            url = url.strip()
            if url.startswith('http://'):
                return url
        return None

    df_clean['image'] = df_clean['imageURLs'].apply(extract_first_image)

    # Contar imágenes válidas
    valid_images = df_clean['image'].notna().sum()
    df_clean = df_clean.dropna(subset=['image'])
    print(f"URLs válidas extraídas: {len(df_clean):,}")
    print(f"Imágenes HTTPS: {df_clean['image'].str.startswith('https').sum():,}")


    # ==================== PASO 4: CREAR ID ÚNICO DE PRODUCTO ====================
    print("\nPaso 4: Generar IDs únicos de productos")

    # Usar columna 'id' si existe, sino crear hash del nombre
    if 'id' in df_clean.columns:
        df_clean['product_id'] = df_clean['id']
    else:
        # Crear ID basado en hash del nombre (más robusto que solo nombre)
        df_clean['product_id'] = df_clean['name'].apply(
            lambda x: abs(hash(str(x).strip().lower())) % (10 ** 10)
        )

    unique_products = df_clean['product_id'].nunique()
    print(f"Productos únicos identificados: {unique_products:,}")


    # ==================== PASO 5: AGRUPAR REVIEWS POR PRODUCTO ====================
    print("\nPaso 5: Agrupar reviews por producto")

    products_dict = defaultdict(lambda: {
        'reviews_texts': [],
        'reviews_ratings': [],
        'reviews_titles': [],
        'reviews_usernames': []
    })

    # Procesar con barra de progreso
    chunk_size = 10000
    total_chunks = (len(df_clean) // chunk_size) + 1

    for chunk_idx in range(total_chunks):
        start_idx = chunk_idx * chunk_size
        end_idx = min((chunk_idx + 1) * chunk_size, len(df_clean))
        chunk = df_clean.iloc[start_idx:end_idx]

        for idx, row in chunk.iterrows():
            product_id = row['product_id']

            # Primera vez: guardar info básica del producto
            if 'name' not in products_dict[product_id]:
                products_dict[product_id]['name'] = row['name']
                products_dict[product_id]['brand'] = row.get('brand', 'Unknown')
                products_dict[product_id]['image'] = row['image']
                products_dict[product_id]['categories'] = row.get('categories', '')
                products_dict[product_id]['primaryCategories'] = row.get('primaryCategories', '')
                products_dict[product_id]['asins'] = row.get('asins', '')

            # Agregar review si existe
            review_text = row.get('reviews.text')
            if pd.notna(review_text) and str(review_text).strip():
                products_dict[product_id]['reviews_texts'].append(str(review_text))
                products_dict[product_id]['reviews_ratings'].append(
                    float(row.get('reviews.rating', 0))
                )
                products_dict[product_id]['reviews_titles'].append(
                    str(row.get('reviews.title', ''))
                )
                products_dict[product_id]['reviews_usernames'].append(
                    str(row.get('reviews.username', ''))
                )

        # Progreso
        if (chunk_idx + 1) % 10 == 0 or chunk_idx == total_chunks - 1:
            progress = ((chunk_idx + 1) / total_chunks) * 100
            print(f"   Progreso: {progress:.1f}% ({end_idx:,}/{len(df_clean):,} filas)", end='\r')

    print(f"\nAgrupación completada: {len(products_dict):,} productos únicos")


    # ==================== PASO 6: CONSTRUIR DATAFRAME DE PRODUCTOS ====================
    print("\nPaso 6: Construir DataFrame de productos")

    products_list = []

    for product_id, info in products_dict.items():
        # Filtrar productos con pocas reviews
        num_reviews = len(info['reviews_texts'])
        if num_reviews < min_reviews_per_product:
            continue

        # Calcular estadísticas de ratings
        ratings = [r for r in info['reviews_ratings'] if r > 0]
        avg_rating = np.mean(ratings) if ratings else 0

        # Concatenar todas las reviews (limitado para memoria)
        # Tomar máximo 10 reviews más relevantes (las primeras)
        max_reviews_to_keep = 10
        reviews_to_keep = info['reviews_texts'][:max_reviews_to_keep]
        all_reviews_text = ' | '.join(reviews_to_keep)

        # Resumen: top 3 reviews para descripción
        reviews_summary = ' '.join(info['reviews_texts'][:3])

        # Extraer categoría principal
        categories_str = str(info['categories'])
        main_category = 'General'
        if categories_str and categories_str != 'nan':
            cats = categories_str.split(',')
            main_category = cats[0].strip() if cats else 'General'

        products_list.append({
            'product_id': product_id,
            'name': info['name'],
            'brand': info['brand'],
            'image': info['image'],
            'main_category': main_category,
            'categories': info['categories'],
            'asins': info['asins'],
            'num_reviews': num_reviews,
            'avg_rating': round(avg_rating, 2),
            'all_reviews': all_reviews_text,
            'reviews_summary': reviews_summary
        })

    df_products = pd.DataFrame(products_list)
    print(f"DataFrame construido: {len(df_products):,} productos")


    # ==================== PASO 7: CREAR DESCRIPCIONES ENRIQUECIDAS ====================
    print("\nPaso 7: Crear descripciones enriquecidas")

    def clean_text(text):
        """Limpia y normaliza texto para CLIP"""
        if pd.isna(text) or not str(text).strip():
            return ""
        text = str(text)
        # Remover HTML tags
        text = re.sub(r'<[^>]+>', '', text)
        # Remover URLs
        text = re.sub(r'http[s]?://\S+', '', text)
        # Remover caracteres especiales excesivos
        text = re.sub(r'[^\w\s.,!?-]', ' ', text)
        # Remover múltiples espacios
        text = re.sub(r'\s+', ' ', text)
        # Limitar longitud (CLIP tiene límite de tokens)
        return text.strip()[:500]

    # Descripción combinada optimizada para embeddings
    df_products['description'] = df_products.apply(
        lambda row: f"{clean_text(row['name'])} {clean_text(row['brand'])} "
                   f"{clean_text(row['main_category'])} "
                   f"{clean_text(row['reviews_summary'])}",
        axis=1
    )

    print(f"Descripciones creadas")


    # ==================== PASO 8: SELECCIÓN Y ORDENAMIENTO FINAL ====================
    print("\nPaso 8: Selección final de productos")

    # Ordenar por popularidad y calidad
    # Criterio: num_reviews (60%) + avg_rating (40%)
    df_products['popularity_score'] = (
        0.6 * (df_products['num_reviews'] / df_products['num_reviews'].max()) +
        0.4 * (df_products['avg_rating'] / 5.0)
    )

    df_products = df_products.sort_values('popularity_score', ascending=False)

    # Limitar a max_products
    if len(df_products) > max_products:
        df_products = df_products.head(max_products)
        print(f"Seleccionados top {max_products:,} productos más relevantes")
    else:
        print(f"Total de productos: {len(df_products):,}")

    # Resetear índice
    df_products = df_products.reset_index(drop=True)

    # Añadir precio estimado basado en categoría
    category_price_ranges = {
        'Electronics': (50, 500),
        'Computers': (100, 1500),
        'Home': (20, 200),
        'Beauty': (10, 100),
        'Sports': (15, 300),
        'Clothing': (15, 150)
    }

    def estimate_price(category):
        """Estima precio basado en categoría"""
        for cat_name, (min_p, max_p) in category_price_ranges.items():
            if cat_name.lower() in category.lower():
                return f"${np.random.randint(min_p, max_p)}"
        return f"${np.random.randint(20, 200)}"

    df_products['price'] = df_products['main_category'].apply(estimate_price)


    # ==================== RESUMEN FINAL ====================
    print("\n" + "=" * 70)
    print("PREPROCESAMIENTO COMPLETADO")
    print(f"\nRESUMEN:")
    print(f"   • Total productos únicos: {len(df_products):,}")
    print(f"   • Reviews totales procesadas: {df_products['num_reviews'].sum():,}")
    print(f"   • Reviews por producto (promedio): {df_products['num_reviews'].mean():.1f}")
    print(f"   • Reviews por producto (mediana): {df_products['num_reviews'].median():.0f}")
    print(f"   • Rating promedio general: {df_products['avg_rating'].mean():.2f}")
    print(f"   • Productos con 5+ reviews: {(df_products['num_reviews'] >= 5).sum():,}")

    print(f"\nTOP 10 CATEGORÍAS:")
    cat_counts = df_products['main_category'].value_counts().head(10)
    for cat, count in cat_counts.items():
        print(f"   • {cat}: {count:,} productos")

    print(f"\nDISTRIBUCIÓN DE RATINGS:")
    for rating in [5, 4, 3, 2, 1]:
        count = ((df_products['avg_rating'] >= rating - 0.5) &
                 (df_products['avg_rating'] < rating + 0.5)).sum()
        print(f"   • {rating} estrellas: {count:,} productos")

    print("\n" + "=" * 70)

    return df_products


In [31]:
# ==================== EJECUCIÓN ====================
# Usar la ruta donde descargaste el dataset
products_df = preprocess_amazon_reviews_all_files(
    path=path,                    # Ruta del kagglehub
    max_products=5000,            # Máximo 5000 productos
    min_reviews_per_product=1    # Al menos 1 review
)

# Verificar resultado
print("\nMUESTRA DE PRODUCTOS PROCESADOS:")
print(products_df[['name', 'brand', 'main_category', 'num_reviews', 'avg_rating']].head(10))

# Guardar dataset procesado (recomendado)
output_file = 'amazon_products_processed_full.csv'
products_df.to_csv(output_file, index=False)
print(f"\nDataset guardado en '{output_file}'")
print(f"Tamaño del archivo: {os.path.getsize(output_file) / (1024**2):.2f} MB")

PREPROCESAMIENTO DE MÚLTIPLES ARCHIVOS

Paso 1: Cargando todos los archivos CSV...
   Archivos encontrados: ['Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv', '1429_1.csv', 'Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv']

   [1/3] Cargando Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv... 5,000 filas
   [2/3] Cargando 1429_1.csv... 34,660 filas
   [3/3] Cargando Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv... 28,332 filas

 Combinando 3 DataFrames...
 Total combinado: 67,992 filas

Paso 2: Limpieza básica
 Duplicados eliminados: 95

 Columnas disponibles: ['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand', 'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer', 'manufacturerNumber', 'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen', 'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs', 

## 5. Codificación e Indexación Multimodal

Transformación de los productos en embeddings multimodales. Preparar para analizar la calidad del espacio vectorial y cómo se relacionan las representaciones visuales con las textuales en el índice de ChromaDB.

##### Codifica textos usando CLIP text encoder.

In [32]:
def encode_texts(texts, batch_size=32, max_length=77):
    """
    Parámetros:
    -----------
    texts : list of str
        Lista de descripciones de productos a codificar
    batch_size : int
        Tamaño del batch para procesamiento eficiente
    max_length : int
        Longitud máxima de tokens (CLIP límite = 77)

    Retorna:
    --------
    numpy.ndarray
        Array de embeddings de forma (n_texts, 512)
    """
    print(f"\nCodificar {len(texts)} descripciones de texto")

    embeddings = []
    num_batches = (len(texts) + batch_size - 1) // batch_size

    # Procesar en batches con barra de progreso
    for i in tqdm(range(0, len(texts), batch_size), desc="Texto"):
        batch = texts[i:i + batch_size]

        # Tokenizar texto (CLIP maneja truncamiento automático)
        # truncate=True: corta texto si excede 77 tokens
        tokens = clip.tokenize(batch, truncate=True).to(device)

        # Codificar sin calcular gradientes (más rápido)
        with torch.no_grad():
            # model.encode_text() retorna embeddings de 512 dims
            text_features = model.encode_text(tokens)

            # CRÍTICO: Normalizar a norma unitaria
            # Esto permite usar similitud coseno = producto punto
            text_features /= text_features.norm(dim=-1, keepdim=True)

        # Mover a CPU y convertir a numpy
        embeddings.append(text_features.cpu().numpy())

    # Concatenar todos los batches
    all_embeddings = np.vstack(embeddings)

    print(f"   Embeddings de texto: {all_embeddings.shape}")
    print(f"   Norma promedio: {np.linalg.norm(all_embeddings, axis=1).mean():.4f}")

    return all_embeddings

####  Codifica imágenes desde URLs usando CLIP image encoder.


In [60]:
def encode_images(image_urls, batch_size=16, timeout=5):
    """
    Parámetros:
    -----------
    image_urls : list of str
        Lista de URLs o rutas locales de imágenes de productos.
    batch_size : int, opcional
        Tamaño de batch lógico (NO implementado aún, se procesa imagen por imagen).
        Se deja como parámetro para futura optimización.
    timeout : int, opcional
        Tiempo máximo de espera para descargar una imagen desde una URL (en segundos).

    Retorna:
    --------
    tuple:
        - all_embeddings : np.ndarray
            Array de embeddings de forma (n_imágenes, 512).
            Las imágenes fallidas se representan con vectores de ceros.
        - valid_indices : list of int
            Índices de las imágenes procesadas exitosamente.
        - failed_indices : list of int
            Índices de las imágenes que fallaron (descarga, formato, etc.).
    """

    print(f"\nCodificar {len(image_urls)} imágenes de productos")

    embeddings = []
    valid_indices = []
    failed_indices = []

    # Header para evitar bloqueos (Amazon, BestBuy, etc.)
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/91.0.4472.124 Safari/537.36'
        )
    }

    for i in tqdm(range(len(image_urls)), desc="Imágenes"):
        url = image_urls[i]

        try:
            # CARGA DE LA IMAGEN
            if isinstance(url, str) and url.startswith('http'):
                response = requests.get(url, timeout=timeout, headers=headers)
                response.raise_for_status()
                image = Image.open(BytesIO(response.content)).convert('RGB')

            elif isinstance(url, str) and os.path.exists(url):
                image = Image.open(url).convert('RGB')

            else:
                raise ValueError(f"URL o ruta inválida: {url}")

            # PREPROCESAMIENTO (CLIP)
            # preprocess realiza:
            # 1. Resize a 224x224
            # 2. Center crop
            # 3. Normalización estándar CLIP
            # 4. Conversión a tensor
            image_input = preprocess(image).unsqueeze(0).to(device)


            # CODIFICACIÓN
            with torch.no_grad():
                image_features = model.encode_image(image_input)

                # NORMALIZACIÓN (CRÍTICA)
                # Garantiza compatibilidad con embeddings de texto
                image_features /= image_features.norm(dim=-1, keepdim=True)

            embeddings.append(image_features.cpu().numpy())
            valid_indices.append(i)

        except Exception as e:
            # FALLBACK EN CASO DE ERROR
            # Se usa un vector de ceros para mantener alineación
            # Alternativa avanzada: embedding promedio del dataset
            embeddings.append(np.zeros((1, 512)))
            failed_indices.append(i)

            # Mostrar errores de forma controlada (evita spam)
            if len(failed_indices) <= 5 or len(failed_indices) % 10 == 1:
                print(f" Error en imagen {i}: {str(e)[:60]}")

    all_embeddings = np.vstack(embeddings)

    success_rate = (len(valid_indices) / len(image_urls)) * 100
    avg_norm = np.linalg.norm(all_embeddings, axis=1).mean()

    print(f"\n Embeddings de imágenes: {all_embeddings.shape}")
    print(f"   Imágenes exitosas: {len(valid_indices)} ({success_rate:.1f}%)")
    print(f"   Imágenes fallidas: {len(failed_indices)}")
    print(f"   Norma promedio: {avg_norm:.4f}")

    return all_embeddings, valid_indices, failed_indices


####  Combina embeddings de texto e imagen en representación híbrida.

In [34]:
def combine_embeddings(text_embeddings, image_embeddings,
                       text_weight=0.5, image_weight=0.5):
    """
    Parámetros:
    -----------
    text_embeddings : numpy.ndarray
        Embeddings de texto (n, 512)
    image_embeddings : numpy.ndarray
        Embeddings de imagen (n, 512)
    text_weight : float
        Peso del componente textual (0-1)
    image_weight : float
        Peso del componente visual (0-1)

    Retorna:
    --------
    numpy.ndarray
        Embeddings combinados (n, 512), normalizados
    """
    print(f"\nCombinar embeddings (texto: {text_weight}, imagen: {image_weight})...")

    # Promedio ponderado
    combined = text_weight * text_embeddings + image_weight * image_embeddings

    # RE-NORMALIZAR después de combinar (IMPORTANTE)
    # Sin esto, la similitud coseno no funcionaría correctamente
    norms = np.linalg.norm(combined, axis=1, keepdims=True)
    combined_normalized = combined / norms

    print(f"   Embeddings combinados: {combined_normalized.shape}")
    print(f"   Norma promedio: {np.linalg.norm(combined_normalized, axis=1).mean():.4f}")

    return combined_normalized

### INDEXACIÓN CON CHROMADB

##### Crea índice vectorial en ChromaDB para búsqueda eficiente.

In [35]:
def create_vector_index(embeddings, products_df, collection_name="products"):
    """
    Parámetros:
    -----------
    embeddings : numpy.ndarray
        Embeddings multimodales de productos (n, 512)
    products_df : pandas.DataFrame
        DataFrame con información de productos
    collection_name : str
        Nombre de la colección en ChromaDB

    Retorna:
    --------
    chromadb.Collection
        Colección de ChromaDB lista para búsquedas
    """
    print(f"\nCrear índice vectorial '{collection_name}'")

    # Inicializar cliente ChromaDB
    client = chromadb.Client()

    # Eliminar colección existente (si existe)
    try:
        client.delete_collection(collection_name)
        print(f"Colección existente eliminada")
    except:
        pass

    # Crear nueva colección
    # hnsw:space="cosine": Usar similitud coseno
    # HNSW (Hierarchical Navigable Small World): Algoritmo de búsqueda ANN
    collection = client.create_collection(
        name=collection_name,
        metadata={
            "hnsw:space": "cosine",  # Métrica de distancia
            "description": "Amazon products multimodal embeddings"
        }
    )

    print(f"Colección creada con HNSW (similitud coseno)")

    # Preparar datos para indexar
    print(f" Agregar {len(embeddings)} productos al índice")

    # Convertir embeddings a lista (ChromaDB requiere este formato)
    embeddings_list = embeddings.tolist()

    # Preparar documentos (texto para contexto)
    documents = products_df['description'].tolist()

    # Preparar metadatos (info adicional del producto)
    metadatas = []
    for _, row in products_df.iterrows():
        metadatas.append({
            'name': str(row['name'])[:500],  # Limitar longitud
            'brand': str(row.get('brand', 'Unknown'))[:100],
            'image': str(row['image'])[:500],
            'category': str(row.get('main_category', 'General'))[:100],
            'price': str(row.get('price', 'N/A'))[:20],
            'num_reviews': int(row.get('num_reviews', 0)),
            'avg_rating': float(row.get('avg_rating', 0.0))
        })

    # Crear IDs únicos
    ids = [f"product_{i}" for i in range(len(products_df))]

    # AGREGAR AL ÍNDICE EN BATCHES (ChromaDB tiene límite por request)
    batch_size = 1000
    num_batches = (len(embeddings) + batch_size - 1) // batch_size

    for i in tqdm(range(0, len(embeddings), batch_size), desc="Indexando"):
        end_idx = min(i + batch_size, len(embeddings))

        collection.add(
            embeddings=embeddings_list[i:end_idx],
            documents=documents[i:end_idx],
            metadatas=metadatas[i:end_idx],
            ids=ids[i:end_idx]
        )

    # Verificar índice
    count = collection.count()
    print(f"\n   Índice creado exitosamente")
    print(f"   Total productos indexados: {count:,}")

    # Estadísticas del índice
    print(f"\nÍNDICE:")
    print(f"   • Dimensión de embeddings: 512")
    print(f"   • Algoritmo: HNSW (Approximate Nearest Neighbors)")
    print(f"   • Métrica de similitud: Coseno")
    print(f"   • Complejidad búsqueda: O(log n)")
    print(f"   • Memoria estimada: {(count * 512 * 4) / (1024**2):.2f} MB")

    return collection

###### EJECUCIÓN COMPLETA

In [61]:
print("CODIFICACIÓN E INDEXACIÓN MULTIMODAL")

# PASO 1: Codificar textos
text_embeddings = encode_texts(
    products_df['description'].tolist(),
    batch_size=32
)

# PASO 2: Codificar imágenes
image_embeddings, valid_img_indices, failed_img_indices = encode_images(
    products_df['image'].tolist(),
    batch_size=16,
    timeout=5
)

# PASO 3: Combinar embeddings
# Estrategia 1: Promedio simple (50-50)
combined_embeddings = combine_embeddings(
    text_embeddings,
    image_embeddings,
    text_weight=0.5,
    image_weight=0.5
)

# ALTERNATIVA: Pesar más el texto (mejor para queries textuales)
# combined_embeddings = combine_embeddings(
#     text_embeddings,
#     image_embeddings,
#     text_weight=0.7,
#     image_weight=0.3
# )

# PASO 4: Crear índice vectorial
collection = create_vector_index(
    combined_embeddings,
    products_df,
    collection_name="products"
)

# ==================== ANÁLISIS DEL ESPACIO VECTORIAL ====================
print("\nANÁLISIS DEL ESPACIO VECTORIAL")

# Calcular similitud promedio entre productos
sample_size = min(100, len(combined_embeddings))
sample_embeddings = combined_embeddings[:sample_size]

# Matriz de similitudes (100x100)
similarity_matrix = np.dot(sample_embeddings, sample_embeddings.T)

# Estadísticas
avg_similarity = (similarity_matrix.sum() - sample_size) / (sample_size * (sample_size - 1))
max_similarity = similarity_matrix[np.triu_indices(sample_size, k=1)].max()
min_similarity = similarity_matrix[np.triu_indices(sample_size, k=1)].min()

print(f"\n Similitud entre productos (muestra de {sample_size}):")
print(f"   • Promedio: {avg_similarity:.4f}")
print(f"   • Máxima: {max_similarity:.4f}")
print(f"   • Mínima: {min_similarity:.4f}")
print(f"   • Rango: {max_similarity - min_similarity:.4f}")

# Interpretación
if avg_similarity > 0.7:
    print(f"  Espacio vectorial muy denso (posible colapso semántico)")
elif avg_similarity < 0.3:
    print(f"  Buena separación semántica entre productos")
else:
    print(f"   Separación moderada (esperado para productos similares)")

print("\nINDEXACIÓN COMPLETADA")
print(f"Sistema listo para búsquedas multimodales")
print(f"   • Productos indexados: {len(products_df):,}")
print(f"   • Embeddings: texto + imagen (512 dims)")
print(f"   • Base de datos: ChromaDB con HNSW")
print(f"   • Búsquedas soportadas: texto→productos, imagen→productos")


CODIFICACIÓN E INDEXACIÓN MULTIMODAL

Codificar 69 descripciones de texto


Texto: 100%|██████████| 3/3 [00:00<00:00, 30.37it/s]


   Embeddings de texto: (69, 512)
   Norma promedio: 1.0000

Codificar 69 imágenes de productos


Imágenes:   7%|▋         | 5/69 [00:00<00:05, 11.97it/s]

 Error en imagen 2: cannot identify image file <_io.BytesIO object at 0x7e4eba30
 Error en imagen 3: HTTPSConnectionPool(host='www.barcodable.com', port=443): Ma


...

 Error en imagen 28: HTTPSConnectionPool(host='www.barcodable.com', port=443): Ma
 Error en imagen 30: HTTPSConnectionPool(host='www.barcodable.com', port=443): Ma


Imágenes: 100%|██████████| 69/69 [00:05<00:00, 13.29it/s]


 Error en imagen 66: cannot identify image file <_io.BytesIO object at 0x7e4eb9c5

 Embeddings de imágenes: (69, 512)
   Imágenes exitosas: 58 (84.1%)
   Imágenes fallidas: 11
   Norma promedio: 0.8406

Combinar embeddings (texto: 0.5, imagen: 0.5)...
   Embeddings combinados: (69, 512)
   Norma promedio: 1.0000

Crear índice vectorial 'products'
Colección existente eliminada
Colección creada con HNSW (similitud coseno)
 Agregar 69 productos al índice


Indexando: 100%|██████████| 1/1 [00:00<00:00, 18.98it/s]


   Índice creado exitosamente
   Total productos indexados: 69

ÍNDICE:
   • Dimensión de embeddings: 512
   • Algoritmo: HNSW (Approximate Nearest Neighbors)
   • Métrica de similitud: Coseno
   • Complejidad búsqueda: O(log n)
   • Memoria estimada: 0.13 MB

ANÁLISIS DEL ESPACIO VECTORIAL

 Similitud entre productos (muestra de 69):
   • Promedio: 0.6163
   • Máxima: 0.9866
   • Mínima: 0.2314
   • Rango: 0.7552
   Separación moderada (esperado para productos similares)

INDEXACIÓN COMPLETADA
Sistema listo para búsquedas multimodales
   • Productos indexados: 69
   • Embeddings: texto + imagen (512 dims)
   • Base de datos: ChromaDB con HNSW
   • Búsquedas soportadas: texto→productos, imagen→productos


In [62]:
# Test: Buscar productos similares a "wireless headphones"
test_query = "wireless bluetooth headphones"
tokens = clip.tokenize([test_query]).to(device)

with torch.no_grad():
    query_embedding = model.encode_text(tokens)
    query_embedding /= query_embedding.norm(dim=-1, keepdim=True)

# Buscar en ChromaDB
results = collection.query(
    query_embeddings=query_embedding.cpu().numpy().tolist(),
    n_results=5
)

print("\nTest de búsqueda: 'wireless bluetooth headphones'")
for i, (name, distance) in enumerate(zip(
    results['metadatas'][0],
    results['distances'][0]
), 1):
    similarity = 1 - distance  # ChromaDB retorna distancias
    print(f"{i}. {name['name'][:60]} (sim: {similarity:.3f})")


Test de búsqueda: 'wireless bluetooth headphones'
1. Certified Refurbished Amazon Fire TV with Alexa Voice Remote (sim: 0.571)
2. Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker (sim: 0.549)
3. All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Inclu (sim: 0.548)
4. Kindle E-reader - White, 6 Glare-Free Touchscreen Display, W (sim: 0.541)
5. Amazon Echo (1st Generationcertified) Color:White Free Shipp (sim: 0.539)


Poca RAM:
1. Reducir batch_size
text_embeddings = encode_texts(texts, batch_size=16)  # vs 32
2. Procesar imágenes en CPU (más lento pero menos VRAM)
model_cpu = model.cpu()
3. Usar precisión float16 (mitad de memoria)
text_embeddings = text_embeddings.astype(np.float16)



## 6. Búsqueda Multimodal (Retrieval Inicial)

Implementación de las funcionalidades de búsqueda text-to-product e image-to-product. Se recupera un conjunto inicial (top-k) de productos candidatos, mostrando información básica como imagen, título y categoría

#### BÚSQUEDA TEXTO → PRODUCTOS
---
    Búsqueda de productos usando consulta textual.




In [67]:
def search_by_text(query, collection, top_k=20, return_scores=True):
    """
    Búsqueda de productos usando consulta textual.

    Parámetros:
    -----------
    query : str
        Consulta textual del usuario (ej: "wireless headphones")
    collection : chromadb.Collection
        Colección de ChromaDB con productos indexados
    top_k : int
        Número de resultados a retornar
    return_scores : bool
        Si True, incluye scores de similitud

    Retorna:
    --------
    dict
        Resultados con 'documents', 'metadatas', 'distances', 'ids'
    """
    print(f"\nBúsqueda por texto: '{query}'")

    # PASO 1: Codificar query de texto con CLIP
    # tokenize() convierte texto a tokens que CLIP entiende
    # truncate=True: corta si excede 77 tokens
    tokens = clip.tokenize([query], truncate=True).to(device)

    # PASO 2: Generar embedding del query
    with torch.no_grad():
        # encode_text() retorna vector de 512 dimensiones
        query_embedding = model.encode_text(tokens)

        # CRÍTICO: Normalizar para similitud coseno
        # Sin esto, las distancias no son comparables
        query_embedding /= query_embedding.norm(dim=-1, keepdim=True)

    # PASO 3: Buscar en el índice vectorial
    # ChromaDB usa HNSW para búsqueda aproximada O(log n)
    # query_embeddings debe ser lista de listas: [[emb1], [emb2], ...]
    results = collection.query(
        query_embeddings=query_embedding.cpu().numpy().tolist(),
        n_results=top_k,
        include=['documents', 'metadatas', 'distances']
    )

    # PASO 4: Convertir distancias a scores de similitud
    # ChromaDB retorna distancias coseno (0 = idéntico, 2 = opuesto)
    # Convertimos a similitud (1 = idéntico, -1 = opuesto)
    if return_scores:
        similarities = [1 - dist for dist in results['distances'][0]]
        results['similarities'] = [similarities]

        # Estadísticas de la búsqueda
        avg_sim = np.mean(similarities)
        max_sim = np.max(similarities)
        min_sim = np.min(similarities)

        print(f"   Resultados encontrados: {len(results['metadatas'][0])}")
        print(f"   Similitud promedio: {avg_sim:.3f}")
        print(f"   Similitud máxima: {max_sim:.3f}")
        print(f"   Similitud mínima: {min_sim:.3f}")

    return results


####  Búsqueda de productos usando imagen de referencia

In [63]:
from io import BytesIO

def search_by_image(image_input, collection, top_k=20, return_scores=True):
    """
    Parámetros:
    -----------
    image_input : str or PIL.Image
        Ruta a imagen, URL, o imagen PIL cargada
    collection : chromadb.Collection
        Colección de ChromaDB con productos indexados
    top_k : int
        Número de resultados a retornar
    return_scores : bool
        Si True, incluye scores de similitud

    Retorna:
    --------
    dict or None
        Resultados con 'documents', 'metadatas', 'distances', 'ids'
        None si falla la carga de la imagen
    """
    print(f"   Buscar top-{top_k} productos similares")

    # PASO 1: Cargar imagen con manejo robusto de errores
    try:
        if isinstance(image_input, str):
            # Caso 1: URL de imagen
            if image_input.startswith('http'):
                print(f"Descargar imagen desde URL")

                # Headers para simular navegador (evitar bloqueos)
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
                }

                response = requests.get(
                    image_input,
                    timeout=10,
                    headers=headers,
                    verify=False  # Ignorar SSL errors
                )

                # Verificar que sea una imagen válida
                if response.status_code != 200:
                    raise ValueError(f"HTTP {response.status_code}: No se pudo descargar la imagen")

                # Verificar content-type
                content_type = response.headers.get('content-type', '')
                if 'image' not in content_type.lower():
                    print(f" Advertencia: Content-Type es '{content_type}', esperaba 'image/*'")

                # Intentar abrir la imagen
                image_bytes = BytesIO(response.content)
                image = Image.open(image_bytes).convert('RGB')

            # Caso 2: Ruta local
            elif os.path.exists(image_input):
                print(f"Cargar imagen desde archivo local")
                image = Image.open(image_input).convert('RGB')

            else:
                raise ValueError(f"Imagen no encontrada: {image_input}")

        # Caso 3: Objeto PIL.Image ya cargado
        elif isinstance(image_input, Image.Image):
            image = image_input.convert('RGB')

        else:
            raise ValueError(f"Tipo de input no soportado: {type(image_input)}")

        # Verificar tamaño mínimo (evitar imágenes corruptas de 1x1)
        if image.size[0] < 10 or image.size[1] < 10:
            raise ValueError(f"Imagen muy pequeña: {image.size}")

        print(f" Imagen cargada exitosamente: {image.size}")

    except requests.exceptions.Timeout:
        print(f"Timeout: La imagen tardó demasiado en descargar")
        return None

    except requests.exceptions.RequestException as e:
        print(f"Error de red: {str(e)[:100]}")
        return None

    except Image.UnidentifiedImageError:
        print(f"    Error: No se pudo identificar el formato de la imagen")
        print(f"    La URL puede no contener una imagen válida")
        return None

    except Exception as e:
        print(f"    Error inesperado al cargar imagen: {str(e)[:100]}")
        return None

    # PASO 2: Preprocesar imagen para CLIP
    try:
        image_tensor = preprocess(image).unsqueeze(0).to(device)
    except Exception as e:
        print(f"    Error al preprocesar imagen: {e}")
        return None

    # PASO 3: Generar embedding de la imagen
    try:
        with torch.no_grad():
            query_embedding = model.encode_image(image_tensor)
            query_embedding /= query_embedding.norm(dim=-1, keepdim=True)
    except Exception as e:
        print(f"    Error al generar embedding: {e}")
        return None

    # PASO 4: Buscar en el índice vectorial
    try:
        results = collection.query(
            query_embeddings=query_embedding.cpu().numpy().tolist(),
            n_results=top_k,
            include=['documents', 'metadatas', 'distances']
        )
    except Exception as e:
        print(f"    Error al buscar en el índice: {e}")
        return None

    # PASO 5: Convertir distancias a similitudes
    if return_scores and results:
        similarities = [1 - dist for dist in results['distances'][0]]
        results['similarities'] = [similarities]

        avg_sim = np.mean(similarities)
        max_sim = np.max(similarities)

        print(f"    Resultados encontrados: {len(results['metadatas'][0])}")
        print(f"    Similitud promedio: {avg_sim:.3f}")
        print(f"    Similitud máxima: {max_sim:.3f}")

    return results

#### Búsqueda híbrida combinando texto e imagen

In [64]:
def search_multimodal(text_query=None, image_input=None, collection=None,
                     top_k=20, text_weight=0.6, image_weight=0.4):
    """
    Parámetros:
    -----------
    text_query : str, optional
        Consulta textual
    image_input : str or PIL.Image, optional
        Imagen de referencia
    collection : chromadb.Collection
        Colección de ChromaDB
    top_k : int
        Número de resultados
    text_weight : float
        Peso del componente textual (0-1)
    image_weight : float
        Peso del componente visual (0-1)

    Retorna:
    --------
    dict
        Resultados combinados
    """
    print(f"\nBúsqueda multimodal híbrida")
    print(f"   Pesos: Texto={text_weight}, Imagen={image_weight}")

    embeddings_to_combine = []

    # PASO 1: Codificar texto si se proporciona
    if text_query:
        print(f"   Procesar query de texto: '{text_query}'")
        try:
            tokens = clip.tokenize([text_query], truncate=True).to(device)
            with torch.no_grad():
                text_emb = model.encode_text(tokens)
                text_emb /= text_emb.norm(dim=-1, keepdim=True)
            embeddings_to_combine.append((text_emb, text_weight))
            print(f"   Embedding de texto generado")
        except Exception as e:
            print(f"   Error al procesar texto: {e}")

    # PASO 2: Codificar imagen si se proporciona
    if image_input:
        print(f"    Procesar imagen de referencia")

        try:
            # Cargar imagen con manejo de errores
            if isinstance(image_input, str):
                if image_input.startswith('http'):
                    headers = {'User-Agent': 'Mozilla/5.0'}
                    response = requests.get(image_input, timeout=10, headers=headers, verify=False)

                    if response.status_code != 200:
                        raise ValueError(f"HTTP {response.status_code}")

                    image = Image.open(BytesIO(response.content)).convert('RGB')
                else:
                    image = Image.open(image_input).convert('RGB')
            else:
                image = image_input.convert('RGB')

            # Verificar tamaño
            if image.size[0] < 10 or image.size[1] < 10:
                raise ValueError(f"Imagen muy pequeña: {image.size}")

            # Codificar
            image_tensor = preprocess(image).unsqueeze(0).to(device)
            with torch.no_grad():
                image_emb = model.encode_image(image_tensor)
                image_emb /= image_emb.norm(dim=-1, keepdim=True)
            embeddings_to_combine.append((image_emb, image_weight))
            print(f"    Embedding de imagen generado")

        except Exception as e:
            print(f"     No se pudo procesar la imagen: {str(e)[:100]}")
            print(f"    Continuando solo con búsqueda por texto...")

    # PASO 3: Verificar que tengamos al menos un embedding
    if not embeddings_to_combine:
        raise ValueError("No se pudo generar ningún embedding (texto ni imagen)")

    # PASO 4: Combinar embeddings con pesos
    if len(embeddings_to_combine) == 1:
        # Solo tenemos un tipo de embedding
        combined_emb = embeddings_to_combine[0][0]
        print(f"    Usando solo {'texto' if text_query else 'imagen'}")
    else:
        # Combinar ambos
        combined_emb = sum(emb * weight for emb, weight in embeddings_to_combine)
        # Re-normalizar después de combinar
        combined_emb /= combined_emb.norm(dim=-1, keepdim=True)

    # PASO 5: Buscar en el índice
    results = collection.query(
        query_embeddings=combined_emb.cpu().numpy().tolist(),
        n_results=top_k,
        include=['documents', 'metadatas', 'distances']
    )

    # Agregar similitudes
    similarities = [1 - dist for dist in results['distances'][0]]
    results['similarities'] = [similarities]

    print(f"    Búsqueda completada: {len(results['metadatas'][0])} resultados")

    return results

#### Encuentra una URL de imagen válida en el dataset.

In [73]:
def get_valid_image_url(df, category_filter=None, max_attempts=10):
    """
    Encuentra una URL de imagen válida en el dataset.

    Parámetros:
    -----------
    df : pandas.DataFrame
        DataFrame de productos
    category_filter : str, optional
        Filtrar por categoría (ej: "Computer", "Electronics")
    max_attempts : int
        Número máximo de intentos

    Retorna:
    --------
    str or None
        URL de imagen válida, o None si no se encuentra
    """
    print(f"\nBuscar imagen válida en el dataset")

    # Filtrar por categoría si se especifica
    if category_filter:
        df_filtered = df[df['main_category'].str.contains(category_filter, case=False, na=False)]
        if len(df_filtered) == 0:
            print(f"   No se encontraron productos en categoría '{category_filter}'")
            df_filtered = df
    else:
        df_filtered = df

    # Intentar cargar imágenes hasta encontrar una válida
    for attempt in range(min(max_attempts, len(df_filtered))):
        row = df_filtered.iloc[attempt]
        image_url = row['image']

        try:
            # Intentar descargar y abrir
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(image_url, timeout=5, headers=headers, verify=False)

            if response.status_code == 200:
                image = Image.open(BytesIO(response.content)).convert('RGB')

                # Verificar tamaño
                if image.size[0] >= 10 and image.size[1] >= 10:
                    print(f"    Imagen válida encontrada: {row['name'][:50]}...")
                    print(f"    URL: {image_url[:80]}...")
                    return image_url

        except Exception as e:
            continue

    print(f"    No se encontró ninguna imagen válida después de {max_attempts} intentos")
    return None

#### Función de visualización

In [70]:
def display_results(results, top_n=10, show_details=True):
    """
    Muestra resultados de búsqueda de forma legible.

    Parámetros:
    -----------
    results : dict
        Resultados de búsqueda
    top_n : int
        Número de resultados a mostrar
    show_details : bool
        Si True, muestra detalles adicionales
    """
    print(f"\n{'='*80}")
    print(f"TOP {top_n} RESULTADOS")
    print(f"{'='*80}\n")

    metadatas = results['metadatas'][0][:top_n]
    similarities = results.get('similarities', [[]])[0][:top_n]

    for i, (meta, sim) in enumerate(zip(metadatas, similarities), 1):
        # Encabezado del producto
        print(f"[{i}] {meta['name'][:70]}")

        # Barra de similitud visual
        sim_percentage = int(sim * 100)
        bar_length = 30
        filled = int(bar_length * sim)
        bar = '█' * filled + '░' * (bar_length - filled)
        print(f"    Similitud: {bar} {sim:.3f} ({sim_percentage}%)")

        # Detalles adicionales
        if show_details:
            print(f"     Marca: {meta['brand']}")
            print(f"     Categoría: {meta['category']}")
            print(f"     Precio: {meta['price']}")
            print(f"     Rating: {meta['avg_rating']:.1f}/5.0 ({meta['num_reviews']} reviews)")
            print(f"     Imagen: {meta['image'][:60]}...")

        print()



##### DEMOSTRACIÓN DE BÚSQUEDA MULTIMODAL

In [71]:
# EJEMPLO 1: Búsqueda por texto (siempre funciona)
print("\nEJEMPLO 1: Búsqueda por texto")

query = "wireless bluetooth headphones noise canceling"
results_text = search_by_text(query, collection, top_k=10)
if results_text:
    display_results(results_text, top_n=5)


EJEMPLO 1: Búsqueda por texto

Búsqueda por texto: 'wireless bluetooth headphones noise canceling'
   Resultados encontrados: 10
   Similitud promedio: 0.552
   Similitud máxima: 0.594
   Similitud mínima: 0.533

TOP 5 RESULTADOS

[1] Certified Refurbished Amazon Fire TV with Alexa Voice Remote
    Similitud: █████████████████░░░░░░░░░░░░░ 0.594 (59%)
     Marca: Amazon
     Categoría: Amazon SMP
     Precio: $44
     Rating: 2.8/5.0 (5 reviews)
     Imagen: https://www.upccodesearch.com/images/barcode/0848719063264.p...

[2] All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Specia
    Similitud: ████████████████░░░░░░░░░░░░░░ 0.559 (55%)
     Marca: Amazon
     Categoría: Fire Tablets
     Precio: $118
     Rating: 4.6/5.0 (112 reviews)
     Imagen: https://www.upccodesearch.com/images/barcode/0841667105758.p...

[3] Kindle E-reader - White, 6 Glare-Free Touchscreen Display, Wi-Fi - Inc
    Similitud: ████████████████░░░░░░░░░░░░░░ 0.554 (55%)
     Marca: Amazon
     Ca

In [74]:
# EJEMPLO 2: Búsqueda por imagen (con validación)
print("\nEJEMPLO 2: Búsqueda por imagen")

# Buscar una imagen válida
valid_image_url = get_valid_image_url(products_df, max_attempts=20)

if valid_image_url:
    results_image = search_by_image(valid_image_url, collection, top_k=10)
    if results_image:
        display_results(results_image, top_n=5)
else:
    print("  Saltando ejemplo de búsqueda por imagen (no hay imágenes válidas)")


EJEMPLO 2: Búsqueda por imagen

Buscar imagen válida en el dataset
    Imagen válida encontrada: AmazonBasics AAA Performance Alkaline Batteries (3...
    URL: https://images-na.ssl-images-amazon.com/images/I/81qmNyJo%2BkL._SL1500_.jpg...
   Buscar top-10 productos similares
Descargar imagen desde URL
 Imagen cargada exitosamente: (1500, 893)
    Resultados encontrados: 10
    Similitud promedio: 0.608
    Similitud máxima: 0.818

TOP 5 RESULTADOS

[1] AmazonBasics AAA Performance Alkaline Batteries (36 Count)
    Similitud: ████████████████████████░░░░░░ 0.818 (81%)
     Marca: Amazonbasics
     Categoría: AA
     Precio: $128
     Rating: 4.5/5.0 (8343 reviews)
     Imagen: https://images-na.ssl-images-amazon.com/images/I/81qmNyJo%2B...

[2] AmazonBasics AA Performance Alkaline Batteries (48 Count) - Packaging 
    Similitud: ████████████████████░░░░░░░░░░ 0.691 (69%)
     Marca: Amazonbasics
     Categoría: AA
     Precio: $46
     Rating: 4.5/5.0 (3728 reviews)
     Imagen: https:

In [75]:
# EJEMPLO 3: Búsqueda híbrida (con fallback)
print("\n EJEMPLO 3: Búsqueda híbrida (texto + imagen)")

query_text = "gaming laptop high performance"

# Buscar imagen de laptop válida
laptop_image_url = get_valid_image_url(products_df, category_filter="Computer", max_attempts=10)

if laptop_image_url:
    results_hybrid = search_multimodal(
        text_query=query_text,
        image_input=laptop_image_url,
        collection=collection,
        top_k=10,
        text_weight=0.7,  # Más peso al texto (más robusto)
        image_weight=0.3
    )
    if results_hybrid:
        display_results(results_hybrid, top_n=5)
else:
    print("    No se encontró imagen válida, usando solo texto...")
    results_hybrid = search_by_text(query_text, collection, top_k=10)
    if results_hybrid:
        display_results(results_hybrid, top_n=5)



 EJEMPLO 3: Búsqueda híbrida (texto + imagen)

Buscar imagen válida en el dataset
    Imagen válida encontrada: Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB,...
    URL: https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5026/5026000_sd.j...

Búsqueda multimodal híbrida
   Pesos: Texto=0.7, Imagen=0.3
   Procesar query de texto: 'gaming laptop high performance'
   Embedding de texto generado
    Procesar imagen de referencia
    Embedding de imagen generado
    Búsqueda completada: 10 resultados

TOP 5 RESULTADOS

[1] Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Blue Kid-Proof Case
    Similitud: ██████████████████████░░░░░░░░ 0.762 (76%)
     Marca: Amazon
     Categoría: Computers
     Precio: $783
     Rating: 4.5/5.0 (1986 reviews)
     Imagen: https://pisces.bbystatic.com/image2/BestBuy_US/images/produc...

[2] Fire Tablet, 7 Display, Wi-Fi, 16 GB - Includes Special Offers, Black
    Similitud: █████████████████████░░░░░░░░░ 0.731 (73%)
     Marca: Amazon
 

###### COMPARACIÓN DE MÉTODOS DE BÚSQUEDA

In [76]:
def compare_search_methods(query_text, query_image=None, collection=None, top_k=5):
    """
    Compara resultados de diferentes métodos de búsqueda.
    """

    # Método 1: Solo texto
    print(f"\n MÉTODO 1: Solo Texto")
    results_text = search_by_text(query_text, collection, top_k=top_k)
    display_results(results_text, top_n=top_k, show_details=False)

    # Método 2: Solo imagen (si se proporciona)
    if query_image:
        print(f"\n  MÉTODO 2: Solo Imagen")
        results_image = search_by_image(query_image, collection, top_k=top_k)
        display_results(results_image, top_n=top_k, show_details=False)

        # Método 3: Híbrido
        print(f"\n MÉTODO 3: Híbrido (60% texto, 40% imagen)")
        results_hybrid = search_multimodal(
            text_query=query_text,
            image_input=query_image,
            collection=collection,
            top_k=top_k,
            text_weight=0.6,
            image_weight=0.4
        )
        display_results(results_hybrid, top_n=top_k, show_details=False)


###### Análisis de calidad de búsqueda

In [77]:
def analyze_search_quality(query, results, expected_category=None):
    """
    Analiza la calidad de los resultados de búsqueda.
    """
    print(f"Query: '{query}'")
    print(f"\nMétricas de calidad:")

    # Similitud promedio
    sims = results['similarities'][0]
    print(f"   • Similitud promedio: {np.mean(sims):.3f}")
    print(f"   • Similitud std: {np.std(sims):.3f}")
    print(f"   • Rango: [{np.min(sims):.3f}, {np.max(sims):.3f}]")

    # Diversidad de categorías
    categories = [m['category'] for m in results['metadatas'][0]]
    unique_cats = len(set(categories))
    print(f"   • Categorías únicas: {unique_cats}/{len(categories)}")

    # Precisión de categoría (si se espera alguna)
    if expected_category:
        matches = sum(1 for cat in categories[:5] if expected_category.lower() in cat.lower())
        precision = matches / 5
        print(f"   • Precisión@5 (categoría '{expected_category}'): {precision:.2%}")

    # Distribución de ratings
    ratings = [m['avg_rating'] for m in results['metadatas'][0]]
    print(f"   • Rating promedio: {np.mean(ratings):.2f}/5.0")

    print()



In [78]:
# Analizar búsqueda de headphones
analyze_search_quality(
    "wireless bluetooth headphones",
    results_text,
    expected_category="Electronics"
)

Query: 'wireless bluetooth headphones'

Métricas de calidad:
   • Similitud promedio: 0.552
   • Similitud std: 0.016
   • Rango: [0.533, 0.594]
   • Categorías únicas: 6/10
   • Precisión@5 (categoría 'Electronics'): 0.00%
   • Rating promedio: 4.05/5.0



###### ESTADÍSTICAS FINALES

In [80]:
# Benchmark de velocidad
import time

# Test de velocidad: 10 búsquedas
test_queries = [
    "laptop",
    "headphones",
    "camera",
    "shoes",
    "watch"
]

print("\n Benchmark de velocidad:")
times = []
for query in test_queries:
    start = time.time()
    _ = search_by_text(query, collection, top_k=20)
    elapsed = time.time() - start
    times.append(elapsed)

avg_time = np.mean(times)
print(f"   • Tiempo promedio por búsqueda: {avg_time*1000:.2f} ms")
print(f"   • Búsquedas por segundo: {1/avg_time:.1f}")

# Capacidad del sistema
print(f"\n Capacidad del sistema:")
print(f"   • Productos indexados: {collection.count():,}")



 Benchmark de velocidad:

Búsqueda por texto: 'laptop'
   Resultados encontrados: 20
   Similitud promedio: 0.502
   Similitud máxima: 0.559
   Similitud mínima: 0.467

Búsqueda por texto: 'headphones'
   Resultados encontrados: 20
   Similitud promedio: 0.476
   Similitud máxima: 0.523
   Similitud mínima: 0.447

Búsqueda por texto: 'camera'
   Resultados encontrados: 20
   Similitud promedio: 0.485
   Similitud máxima: 0.561
   Similitud mínima: 0.462

Búsqueda por texto: 'shoes'
   Resultados encontrados: 20
   Similitud promedio: 0.440
   Similitud máxima: 0.516
   Similitud mínima: 0.404

Búsqueda por texto: 'watch'
   Resultados encontrados: 20
   Similitud promedio: 0.463
   Similitud máxima: 0.531
   Similitud mínima: 0.441
   • Tiempo promedio por búsqueda: 12.12 ms
   • Búsquedas por segundo: 82.5

 Capacidad del sistema:
   • Productos indexados: 69


## 7. Re-ranking de Resultados

Aplicación de un mecanismo de refinamiento mediante un modelo cross-encoder. Permite realizar la comparación cualitativa "antes y después" exigida, analizando críticamente cuándo el re-ranking mejora la relevancia de los productos frente a la búsqueda inicial.

In [84]:
def rerank_results(query, results, top_k=10):
    """
    Re-ranking usando cross-encoder para mejorar relevancia.

    Parámetros:
    -----------
    query : str
        Consulta original del usuario
    results : dict
        Resultados iniciales de ChromaDB
    top_k : int
        Número de resultados finales después del re-ranking

    Retorna:
    --------
    dict
        Resultados reordenados con scores
    """
    print(f"\nAplicar re-ranking con cross-encoder")
    print(f"   Candidatos iniciales: {len(results['documents'][0])}")
    print(f"   Top-k final: {top_k}")

    documents = results['documents'][0]
    metadatas = results['metadatas'][0]

    # Preparar pares query-documento
    pairs = [[query, doc] for doc in documents]

    # Calcular scores con cross-encoder
    print(f"   Calcular scores de relevancia")
    scores = reranker.predict(pairs)

    # Ordenar por score descendente
    ranked_indices = np.argsort(scores)[::-1][:top_k]

    # Reordenar resultados
    reranked_results = {
        'documents': [[documents[i] for i in ranked_indices]],
        'metadatas': [[metadatas[i] for i in ranked_indices]],
        'scores': [scores[i] for i in ranked_indices],
        'original_indices': ranked_indices.tolist()
    }

    # Estadísticas
    avg_score = np.mean(reranked_results['scores'])
    max_score = np.max(reranked_results['scores'])

    print(f"    Re-ranking completado")
    print(f"    Score promedio: {avg_score:.4f}")
    print(f"    Score máximo: {max_score:.4f}")

    return reranked_results


##### Comparación antes/después del re-ranking

In [103]:
query = "laptop for gaming high performance"

initial_results = search_by_text(query, collection, top_k=20)

print(" ANTES del Re-ranking (Top 5):")
for i, meta in enumerate(initial_results['metadatas'][0][:5], 1):
    print(f"{i}. {meta['name']}")

# Re-ranking
reranked = rerank_results(query, initial_results, top_k=5)

print("\nDESPUÉS del Re-ranking (Top 5):")
for i, (meta, score) in enumerate(zip(reranked['metadatas'][0], reranked['scores']), 1):
    print(f"{i}. {meta['name']} (score: {score:.4f})")


Búsqueda por texto: 'laptop for gaming high performance'
   Resultados encontrados: 20
   Similitud promedio: 0.542
   Similitud máxima: 0.610
   Similitud mínima: 0.501
 ANTES del Re-ranking (Top 5):
1. Fire Tablet, 7 Display, Wi-Fi, 16 GB - Includes Special Offers, Black
2. All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Blue
3. Certified Refurbished Amazon Fire TV with Alexa Voice Remote
4. Kindle E-reader - White, 6 Glare-Free Touchscreen Display, Wi-Fi - Includes Special Offers
5. All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Black

Aplicar re-ranking con cross-encoder
   Candidatos iniciales: 20
   Top-k final: 5
   Calcular scores de relevancia
    Re-ranking completado
    Score promedio: -9.9376
    Score máximo: -8.6702

DESPUÉS del Re-ranking (Top 5):
1. Fire Tablet, 7 Display, Wi-Fi, 16 GB - Includes Special Offers, Black (score: -8.6702)
2. Amazon Fire TV Gaming Edition Streaming Media Player (score: -9.581

## 8. Generación Aumentada por Recuperación (RAG)

In [112]:
!pip install -U -q google-generativeai

Construcción de una respuesta justificativa basada en los productos mejor posicionados. El sistema utiliza el contexto recuperado para generar una recomendación grounded (bien fundamentada).

In [125]:
import google.generativeai as genai
from google.colab import userdata

def generate_rag_response_gemini(query, top_products, api_key=None):
    """
    Versión optimizada para modelos Gemini 3 Flash con soporte de precios.
    """
    try:
        if api_key is None:
            api_key = userdata.get('GEMINI_API_KEY')

        genai.configure(api_key=api_key)

        # 1. Normalización de datos
        productos_lista = []
        if isinstance(top_products, dict) and 'metadatas' in top_products:
            productos_lista = top_products['metadatas'][0]
        elif isinstance(top_products, list):
            productos_lista = top_products

        if not productos_lista:
            return "No se encontraron productos en el inventario."

        # 2. Construcción del Contexto CON INFORMACIÓN DE PRECIO
        contexto_text = ""
        for i, p in enumerate(productos_lista[:5], 1):  # Aumentado a 5 productos
            name = p.get('name', 'N/A')
            price = p.get('price', 'N/A')
            brand = p.get('brand', 'N/A')
            category = p.get('category', 'N/A')
            rating = p.get('avg_rating', 'N/A')
            num_reviews = p.get('num_reviews', 0)
            reviews = p.get('contexto_rag', 'Sin reseñas')[:200]

            contexto_text += f"""PRODUCTO {i}: {name}
 PRECIO: {price}
 MARCA: {brand}
 CATEGORÍA: {category}
 RATING: {rating}/5.0 ({num_reviews} reseñas)
 OPINIONES: {reviews}

"""

        # 3. Prompt MEJORADO con instrucciones sobre precios
        prompt = f"""Actúa como un experto asistente de compras.
Responde a la consulta del usuario basándote EXCLUSIVAMENTE en el contexto proporcionado.

IMPORTANTE:
- Si te preguntan por precios, SIEMPRE menciona el precio exacto en tu respuesta
- Si preguntan "cuál es el más barato", compara los precios y recomienda el de menor costo
- Si preguntan "cuál es el más caro", compara y recomienda el de mayor precio
- Usa los ratings y opiniones para justificar tu recomendación
- Si el contexto no tiene la respuesta, admítelo claramente

CONTEXTO DE PRODUCTOS:
{contexto_text}

CONSULTA DEL USUARIO: "{query}"

RESPUESTA JUSTIFICADA (menciona precios cuando sea relevante):"""

        # 4. LLAMADA AL MODELO
        model = genai.GenerativeModel('gemini-3-flash-preview')
        response = model.generate_content(prompt)

        return response.text

    except Exception as e:
        print(f"DEBUG - Error: {e}")
        # Fallback
        return f"Sugerencia rápida: {productos_lista[0].get('name') if productos_lista else 'No hay datos'}"

###### Ejemplo de RAG

In [126]:
query_rag = "wireless bluetooth headphones with noise canceling"
print(f"\n Query: '{query_rag}'")
print()

# Obtener productos (retrieval + re-ranking)
initial = search_by_text(query_rag, collection, top_k=20)
reranked = rerank_results(query_rag, initial, top_k=5)

print(f"Productos recuperados: {len(reranked['metadatas'][0])}")
print(f"Top producto: {reranked['metadatas'][0][0]['name']}\n")

try:
    rag_response_gemini = generate_rag_response_gemini(
        query_rag,
        reranked
        # api_key=GEMINI_API_KEY  # Descomentar si usas key hardcoded
    )

    print(" Respuesta Gemini:")
    print("-" * 80)
    print(rag_response_gemini)


except Exception as e:
    print(f"  No se pudo usar Gemini: {e}")
    print("   Continuando con opción 2...")

print()


 Query: 'wireless bluetooth headphones with noise canceling'


Búsqueda por texto: 'wireless bluetooth headphones with noise canceling'
   Resultados encontrados: 20
   Similitud promedio: 0.526
   Similitud máxima: 0.577
   Similitud mínima: 0.504

Aplicar re-ranking con cross-encoder
   Candidatos iniciales: 20
   Top-k final: 5
   Calcular scores de relevancia
    Re-ranking completado
    Score promedio: -10.5790
    Score máximo: -8.2539
Productos recuperados: 5
Top producto: Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker



ERROR:tornado.access:503 POST /v1beta/models/gemini-3-flash-preview:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 31899.15ms


 Respuesta Gemini:
--------------------------------------------------------------------------------
Lo siento, pero basándome **exclusivamente** en el contexto proporcionado, no hay ningún producto que coincida con tu búsqueda de **"wireless bluetooth headphones with noise canceling"** (auriculares inalámbricos con cancelación de ruido).

La lista de productos disponibles incluye únicamente altavoces inteligentes, tabletas y lectores de libros electrónicos:

1.  **Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker**: $115 (Altavoz portátil).
2.  **Echo Dot (Previous generation)**: $293 (Electrónica de consumo).
3.  **All-new Echo (2nd Generation)**: $135 (Altavoz inteligente).
4.  **All-New Fire HD 8 Tablet**: $59 (Tableta).
5.  **All-New Kindle Oasis E-reader**: $190 (E-reader).

Admito claramente que el contexto no contiene auriculares de ningún tipo.



## 9. Búsqueda Conversacional con Contexto

Implementación de la gestión de estado para conservar el "ancla" de la sesión (producto inicial) y las restricciones acumuladas, como colores o precios. Se permite el refinamiento iterativo (ej: "ahora en color blanco").

In [121]:
class ConversationalSearchSession:
    """
    Maneja el contexto de búsqueda conversacional.
    """
    def __init__(self):
        self.anchor_query = None
        self.constraints = {}
        self.history = []
        self.last_results = None

    def add_turn(self, query, results):
        """Agregar turno a la historia."""
        self.history.append({'query': query, 'results': results})
        if len(self.history) > 3:
            self.history.pop(0)
        self.last_results = results

    def update_constraints(self, query):
        """Extraer restricciones del query."""
        query_lower = query.lower()

        # Detectar colores
        colors = ['red', 'blue', 'white', 'black', 'green', 'rojo', 'azul', 'blanco', 'negro']
        for color in colors:
            if color in query_lower:
                self.constraints['color'] = color

        # Detectar precio
        if any(word in query_lower for word in ['cheap', 'barato', 'budget']):
            self.constraints['price'] = 'low'
        elif any(word in query_lower for word in ['expensive', 'premium', 'caro']):
            self.constraints['price'] = 'high'

    def build_contextual_query(self, current_query):
        """Construir query enriquecido con contexto."""
        self.update_constraints(current_query)

        enriched = current_query

        # Agregar ancla
        if self.anchor_query and self.anchor_query.lower() not in current_query.lower():
            enriched = f"{self.anchor_query} {current_query}"

        # Agregar restricciones
        for constraint, value in self.constraints.items():
            if value not in enriched.lower():
                enriched += f" {value}"

        return enriched

    def reset(self):
        """Reiniciar sesión."""
        self.anchor_query = None
        self.constraints = {}
        self.history = []
        self.last_results = None


##### Ejemplo de búsqueda conversacional

In [122]:

session = ConversationalSearchSession()

# Turno 1
query1 = "running shoes"
print(f"Usuario: {query1}")
results1 = search_by_text(query1, collection, top_k=10)
reranked1 = rerank_results(query1, results1, top_k=3)
session.anchor_query = query1
session.add_turn(query1, reranked1)
print(f" Sistema: Top 1 - {reranked1['metadatas'][0][0]['name']}\n")

# Turno 2: Refinamiento
query2 = "in white color"
print(f" Usuario: {query2}")
contextual_query = session.build_contextual_query(query2)
print(f" Query contextual: '{contextual_query}'")
results2 = search_by_text(contextual_query, collection, top_k=10)
reranked2 = rerank_results(contextual_query, results2, top_k=3)
session.add_turn(query2, reranked2)
print(f" Sistema: Top 1 - {reranked2['metadatas'][0][0]['name']}\n")

print(f"Contexto acumulado: {session.constraints}")

Usuario: running shoes

Búsqueda por texto: 'running shoes'
   Resultados encontrados: 10
   Similitud promedio: 0.467
   Similitud máxima: 0.527
   Similitud mínima: 0.433

Aplicar re-ranking con cross-encoder
   Candidatos iniciales: 10
   Top-k final: 3
   Calcular scores de relevancia
    Re-ranking completado
    Score promedio: -11.3032
    Score máximo: -11.2842
 Sistema: Top 1 - All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Blue

 Usuario: in white color
 Query contextual: 'running shoes in white color'

Búsqueda por texto: 'running shoes in white color'
   Resultados encontrados: 10
   Similitud promedio: 0.433
   Similitud máxima: 0.517
   Similitud mínima: 0.399

Aplicar re-ranking con cross-encoder
   Candidatos iniciales: 10
   Top-k final: 3
   Calcular scores de relevancia
    Re-ranking completado
    Score promedio: -10.5345
    Score máximo: -9.3831
 Sistema: Top 1 - Amazon Echo (1st Generationcertified) Color:White Free Shipping

Con

###### GENERAR IMÁGENES DEL PRODUCTO EN DIFERENTES

In [129]:
# Instalación de Stable Diffusion para generación de imágenes
!pip install -U "diffusers<0.25.0" transformers accelerate

# Para mostrar imágenes generadas
!pip install -q matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 38.6 MB/s eta 0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.36.0
    Uninstalling diffusers-0.36.0:
      Successfully uninstalled diffusers-0.36.0


In [130]:
from diffusers import StableDiffusionPipeline
import matplotlib.pyplot as plt

# Variable global para el pipeline (se carga una sola vez)
image_generation_pipeline = None

def initialize_image_generator():
    """
    Inicializa el generador de imágenes (solo una vez).
    """
    global image_generation_pipeline

    if image_generation_pipeline is None:
        print(" Inicializando Stable Diffusion...")
        print(" Esto puede tomar 1-2 minutos la primera vez...")

        # Usar modelo más ligero para Colab
        model_id = "runwayml/stable-diffusion-v1-5"

        image_generation_pipeline = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            safety_checker=None  # Desactivar para productos
        )

        # Mover a GPU si está disponible
        if torch.cuda.is_available():
            image_generation_pipeline = image_generation_pipeline.to("cuda")
            print(" Pipeline cargado en GPU")
        else:
            print(" GPU no disponible, usando CPU (será más lento)")

        print(" Generador de imágenes listo")

    return image_generation_pipeline

def generate_product_image(product_name, color, category="product"):
    """
    Genera una imagen del producto en el color especificado.

    Parámetros:
    -----------
    product_name : str
        Nombre del producto
    color : str
        Color deseado (ej: 'white', 'black', 'red')
    category : str
        Categoría del producto (ej: 'laptop', 'headphones')

    Retorna:
    --------
    PIL.Image
        Imagen generada
    """
    try:
        # Inicializar pipeline si no existe
        pipe = initialize_image_generator()

        # Crear prompt optimizado
        prompt = f"professional product photography of {color} {product_name}, {category}, studio lighting, white background, high quality, detailed, 4k"

        # Prompt negativo para mejor calidad
        negative_prompt = "low quality, blurry, watermark, text, logo, human, person, hands"

        print(f" Generando imagen: {color} {product_name}...")

        # Generar imagen
        with torch.autocast("cuda" if torch.cuda.is_available() else "cpu"):
            image = pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=30,  # Menos pasos para más velocidad
                guidance_scale=7.5,
                width=512,
                height=512
            ).images[0]

        print(" Imagen generada exitosamente")
        return image

    except Exception as e:
        print(f" Error generando imagen: {e}")
        # Crear imagen de placeholder
        placeholder = Image.new('RGB', (512, 512), color='lightgray')
        return placeholder

def display_generated_image(image, title="Producto Generado"):
    """
    Muestra la imagen generada.
    """
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

## 10. Interfaz de Usuario con Gradio

Diseño de la interfaz gráfica conversacional que integra todos los componentes del sistema. Permite al usuario subir imágenes, ingresar texto y visualizar tanto los resultados reordenados como la respuesta justificativa generada por el módulo RAG.

In [ ]:
# Sistema conversacional global
global_session = ConversationalSearchSession()

def search_interface(query_text, query_image, use_reranking, chat_history):
    """
    Interfaz principal de búsqueda multimodal con:
    - Texto / Imagen
    - Contexto conversacional
    - Re-ranking opcional
    - RAG
    - Generación de imágenes por color (cuando el usuario lo solicita)
    """
    try:
        # ============================
        # 1. DETECCIÓN DE INTENCIÓN (imagen por color)
        generate_image = False
        color_requested = None

        query_lower = query_text.lower() if query_text else ""

        color_keywords = {
            'blanco': 'white', 'white': 'white',
            'negro': 'black', 'black': 'black',
            'rojo': 'red', 'red': 'red',
            'azul': 'blue', 'blue': 'blue',
            'verde': 'green', 'green': 'green',
            'gris': 'gray', 'gray': 'gray',
            'rosa': 'pink', 'pink': 'pink'
        }

        image_triggers = [
            'muéstrame', 'mostrar', 'ver', 'generar',
            'show me', 'generate', 'imagen'
        ]

        if query_text and any(t in query_lower for t in image_triggers):
            for keyword, color in color_keywords.items():
                if keyword in query_lower:
                    generate_image = True
                    color_requested = color
                    break

        # ============================
        # 2. DETERMINAR TIPO DE BÚSQUEDA
        # ============================
        if query_image is not None:
            results = search_by_image(query_image, collection, top_k=20)
            search_type = "imagen"
            query_for_rerank = "producto similar a la imagen"

        elif query_text:
            contextual_query = global_session.build_contextual_query(query_text)
            results = search_by_text(contextual_query, top_k=20)
            search_type = "texto"
            query_for_rerank = contextual_query

        else:
            return "Por favor ingresa un texto o sube una imagen.", "", chat_history

        # ============================
        # 3. RE-RANKING
        # ============================
        if use_reranking:
            results = rerank_results(query_for_rerank, results, top_k=10)
            ranking_info = " Re-ranking aplicado"
        else:
            ranking_info = " Sin re-ranking"

        # ============================
        # 4. CONTEXTO CONVERSACIONAL
        # ============================
        global_session.add_turn(query_text or "[imagen]", results)

        # ============================
        # 5. RAG
        # ============================
        rag_response = generate_rag_response_gemini(
            query_text or "productos similares",
            results
        )

        # ============================
        # 6. FORMATEO DE RESULTADOS
        # ============================
        output = f"**🔍 Búsqueda por {search_type}** | {ranking_info}\n\n"

        # ============================
        # 7. GENERACIÓN DE IMAGEN (SI APLICA)
        # ============================
        if generate_image and color_requested and results['metadatas'][0]:
            top_product = results['metadatas'][0][0]

            product_name = top_product['name']
            category = top_product.get('category', 'product')

            output += (
                f" **Generando imagen de '{product_name}' "
                f"en color {color_requested}...**\n\n"
            )

            generated_image = generate_product_image(
                product_name,
                color_requested,
                category
            )

            import os
            temp_dir = "/tmp/generated_images"
            os.makedirs(temp_dir, exist_ok=True)

            image_path = (
                f"{temp_dir}/{color_requested}_"
                f"{product_name[:30].replace(' ', '_')}.png"
            )

            generated_image.save(image_path)

            output += f"Imagen generada: `{image_path}`\n\n"

            rag_response = (
                f" He generado una visualización del producto "
                f"**{product_name}** en color **{color_requested}**.\n\n"
                f"{rag_response}"
            )

        # ============================
        # 8. TOP RESULTADOS
        # ============================
        output += "**Top 5 Resultados:**\n\n"

        for i, meta in enumerate(results['metadatas'][0][:5], 1):
            output += f"**{i}. {meta['name']}**\n"
            output += f"     Categoría: {meta.get('category', 'N/A')}\n"
            output += f"     Precio: {meta.get('price', 'N/A')}\n"

            if 'avg_rating' in meta:
                output += (
                    f"     Rating: {meta['avg_rating']}/5.0 "
                    f"({meta.get('num_reviews', 0)} reseñas)\n"
                )

            if 'scores' in results:
                output += f"     Score: {results['scores'][i-1]:.4f}\n"

            output += "\n"

        # ============================
        # 9. HISTORIAL
        # ============================
        chat_history.append(
            (query_text or "[Imagen subida]", rag_response)
        )

        return output, rag_response, chat_history

    except Exception as e:
        return f" Error: {str(e)}", "", chat_history
def reset_session():
    """Reiniciar sesión conversacional."""
    global_session.reset()
    return [], "Sesión reiniciada. Contexto limpiado."

# Crear interfaz Gradio
with gr.Blocks(title="Sistema Multimodal E-Commerce", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    #  Sistema de Recuperación Multimodal para E-Commerce

    **Búsqueda inteligente con contexto conversacional**

    -  Búsqueda por texto
    -  Búsqueda por imagen
    -  Re-ranking automático
    -  Respuestas generadas con RAG
    -  Contexto conversacional
    """)

    with gr.Row():
        with gr.Column(scale=1):
            query_text = gr.Textbox(
                label="Búsqueda por Texto",
                placeholder="Ej: wireless headphones bluetooth",
                lines=2
            )
            query_image = gr.Image(
                label="Búsqueda por Imagen",
                type="filepath"
            )
            use_reranking = gr.Checkbox(
                label="Aplicar Re-ranking",
                value=True
            )

            with gr.Row():
                search_btn = gr.Button(" Buscar", variant="primary")
                reset_btn = gr.Button(" Reiniciar Sesión")

        with gr.Column(scale=2):
            results_output = gr.Markdown(label="Resultados")
            rag_output = gr.Markdown(label="Recomendación AI")

    gr.Markdown("---")
    gr.Markdown("###  Historial Conversacional")
    chatbot = gr.Chatbot(label="Conversación", height=300)

    # Eventos
    search_btn.click(
        fn=search_interface,
        inputs=[query_text, query_image, use_reranking, chatbot],
        outputs=[results_output, rag_output, chatbot]
    )

    reset_btn.click(
        fn=reset_session,
        outputs=[chatbot, rag_output]
    )

    gr.Markdown("""
    ---
    ** Consejos de uso:**
    - Para refinar búsquedas, usa frases como "en color blanco" o "más barato"
    - El sistema mantiene contexto de los últimos 3 turnos
    - Puedes combinar texto e imagen para búsquedas híbridas
    """)

# Lanzar interfaz
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0ecd8fd8f17771aafb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error

   Buscar top-20 productos similares
Cargar imagen desde archivo local
 Imagen cargada exitosamente: (450, 450)
    Resultados encontrados: 20
    Similitud promedio: 0.675
    Similitud máxima: 0.804

Aplicar re-ranking con cross-encoder
   Candidatos iniciales: 20
   Top-k final: 10
   Calcular scores de relevancia
    Re-ranking completado
    Score promedio: -11.3199
    Score máximo: -10.9565


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error

   Buscar top-20 productos similares
Cargar imagen desde archivo local
 Imagen cargada exitosamente: (450, 450)
    Resultados encontrados: 20
    Similitud promedio: 0.675
    Similitud máxima: 0.804

Aplicar re-ranking con cross-encoder
   Candidatos iniciales: 20
   Top-k final: 10
   Calcular scores de relevancia
    Re-ranking completado
    Score promedio: -11.3199
    Score máximo: -10.9565
   Buscar top-20 productos similares
Cargar imagen desde archivo local
 Imagen cargada exitosamente: (450, 450)
    Resultados encontrados: 20
    Similitud promedio: 0.675
    Similitud máxima: 0.804

Aplicar re-ranking con cross-encoder
   Candidatos iniciales: 20
   Top-k final: 10
   Calcular scores de relevancia
    Re-ranking completado
    Score promedio: -11.3199
    Score máximo: -10.9565


laptop for gaming high performance

## 11. Análisis y Evaluación

Se evalúa la calidad del retrieval, se discuten limitaciones del sistema (como ruido en las imágenes o ambigüedad textual)

In [146]:
# Análisis del impacto del re-ranking
print("\n ANÁLISIS CUALITATIVO DEL SISTEMA\n")


# Casos de prueba
test_queries = [
    "laptop for gaming",
    "wireless bluetooth earbuds",
    "running shoes comfortable",
    "kitchen blender powerful"
]

for query in test_queries:
    print(f"\nQuery: '{query}'")


    # Sin re-ranking
    initial = search_by_text(query,collection, top_k=10)
    print("\nSIN Re-ranking (Top 3):")
    for i, meta in enumerate(initial['metadatas'][0][:3], 1):
        print(f"  {i}. {meta['name'][:60]}")

    # Con re-ranking
    reranked = rerank_results(query, initial, top_k=3)
    print("\nCON Re-ranking (Top 3):")
    for i, (meta, score) in enumerate(zip(reranked['metadatas'][0], reranked['scores']), 1):
        print(f"  {i}. {meta['name'][:60]}... (score: {score:.3f})")

    print("\n" + "=" * 60)

print("\n Análisis completado")


 ANÁLISIS CUALITATIVO DEL SISTEMA


Query: 'laptop for gaming'

Búsqueda por texto: 'laptop for gaming'
   Resultados encontrados: 10
   Similitud promedio: 0.578
   Similitud máxima: 0.620
   Similitud mínima: 0.548

SIN Re-ranking (Top 3):
  1. Fire Tablet, 7 Display, Wi-Fi, 16 GB - Includes Special Offe
  2. All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Inclu
  3. Certified Refurbished Amazon Fire TV with Alexa Voice Remote

Aplicar re-ranking con cross-encoder
   Candidatos iniciales: 10
   Top-k final: 3
   Calcular scores de relevancia
    Re-ranking completado
    Score promedio: -6.0096
    Score máximo: -5.3446

CON Re-ranking (Top 3):
  1. Fire Tablet, 7 Display, Wi-Fi, 16 GB - Includes Special Offe... (score: -5.345)
  2. Amazon Fire TV Gaming Edition Streaming Media Player... (score: -5.742)
  3. Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Blue Kid-... (score: -6.943)


Query: 'wireless bluetooth earbuds'

Búsqueda por texto: 'wireless bluetooth earbuds'
  

## 12. Exportar Resultados para Informe

Se exportan ejemplos de consultas, resultados y el impacto cuantitativo/cualitativo del re-ranking para sustentar la documentación final

In [148]:
# Guardar ejemplos de resultados para el informe
import json

report_data = {
    'corpus_info': {
        'total_products': len(products_df),
        'embedding_dimension': combined_embeddings.shape[1],
        'categories': products_df['main_category'].value_counts().head(10).to_dict() if 'main_category' in products_df else {}
    },
    'example_searches': [],
    'reranking_impact': []
}

# Ejemplos de búsquedas
for query in test_queries[:2]:
    initial = search_by_text(query, collection, top_k=5)
    reranked = rerank_results(query, initial, top_k=5)

    report_data['example_searches'].append({
        'query': query,
        'top_3_products': [
            meta['name'] for meta in reranked['metadatas'][0][:3]
        ]
    })

# Guardar datos
with open('report_data.json', 'w', encoding='utf-8') as f:
    json.dump(report_data, f, indent=2, ensure_ascii=False)

print(" Datos para informe exportados a 'report_data.json'")
print("\nResumen del corpus:")
print(f"  - Total productos: {report_data['corpus_info']['total_products']}")
print(f"  - Dimensión embeddings: {report_data['corpus_info']['embedding_dimension']}")


Búsqueda por texto: 'laptop for gaming'
   Resultados encontrados: 5
   Similitud promedio: 0.597
   Similitud máxima: 0.620
   Similitud mínima: 0.576

Aplicar re-ranking con cross-encoder
   Candidatos iniciales: 5
   Top-k final: 5
   Calcular scores de relevancia
    Re-ranking completado
    Score promedio: -9.8875
    Score máximo: -5.3446

Búsqueda por texto: 'wireless bluetooth earbuds'
   Resultados encontrados: 5
   Similitud promedio: 0.520
   Similitud máxima: 0.548
   Similitud mínima: 0.506

Aplicar re-ranking con cross-encoder
   Candidatos iniciales: 5
   Top-k final: 5
   Calcular scores de relevancia
    Re-ranking completado
    Score promedio: -10.0431
    Score máximo: -5.5686
 Datos para informe exportados a 'report_data.json'

Resumen del corpus:
  - Total productos: 69
  - Dimensión embeddings: 512


## 13. Conclusiones

Este notebook implementa un sistema completo de recuperación multimodal que cumple con todos los requisitos:

 **Indexación multimodal**: CLIP para embeddings de texto e imagen

 **Búsqueda multimodal**: Texto → productos e Imagen → productos

 **Re-ranking**: Cross-encoder para mejorar relevancia

 **RAG**: Generación basada en contexto recuperado

 **Búsqueda conversacional**: Mantenimiento de contexto entre turnos

 **Interfaz de usuario**: Gradio con todas las funcionalidades
